In [31]:
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import pairwise_distances
from time import time
import numpy as np
import torch

In [32]:
x1_path = '../graph_data/pm_node_embeddings.pt'
x2_path = '../graph_data/ot_node_embeddings.pt'
edge_index1_path = '../graph_data/pm_edge_index.pt'
edge_index2_path = '../graph_data/ot_edge_index.pt'

x1 = torch.load(x1_path).float()
x2 = torch.load(x2_path).float()
edge_index1 = torch.load(edge_index1_path)
edge_index2 = torch.load(edge_index2_path)

#### Hungarian algorithm on embeddings

In [34]:
times = []
for i in [10, 100, 300, 500, 1000, 2000, 4000, 8000, 11702]:
    x_pm = x1[:i]
    x_ot = x2[:i]
    sim_eucl = pairwise_distances(x_pm, x_ot)
    sim_cos = pairwise_distances(x_pm, x_ot, metric='cosine')
    start = time()
    match_eucl = linear_sum_assignment(sim_eucl)
    match_cos = linear_sum_assignment(sim_cos)
    end = time()
    times.append(end - start)
    hits_at_1_eucl = (match_eucl[0] == match_eucl[1]).sum() / i
    hits_at_1_cos = (match_cos[0] == match_cos[1]).sum() / i
    
    print(f'{i} eucl: {hits_at_1_eucl:.02f}, cos: {hits_at_1_cos:.02f}, time: {end - start:.02f}')

10 eucl: 1.00, cos: 1.00, time: 0.00
100 eucl: 0.94, cos: 0.94, time: 0.00
300 eucl: 0.83, cos: 0.87, time: 0.01
500 eucl: 0.78, cos: 0.84, time: 0.03
1000 eucl: 0.63, cos: 0.70, time: 0.20
2000 eucl: 0.57, cos: 0.64, time: 1.55
4000 eucl: 0.66, cos: 0.71, time: 13.06
8000 eucl: 0.81, cos: 0.83, time: 70.51
11702 eucl: 0.74, cos: 0.76, time: 127.05


#### Preparing topological features

In [29]:
pm_degree_dict = {}
pm_neigh_dict = {}
pm_features = np.empty((x1.shape[0], 5))

for i in range(x1.shape[0]):
    pm_degree_dict.setdefault(i, 0)
    pm_neigh_dict.setdefault(i, 0)
    
for edge in edge_index1.T:
    s, t = list(map(int, edge))
    pm_degree_dict[s] = pm_degree_dict.setdefault(s, 0) + 1
    pm_degree_dict[t] = pm_degree_dict.setdefault(t, 0) + 1

for edge in edge_index1.T:
    s, t = list(map(int, edge))
    pm_neigh_dict[s] = np.append(pm_neigh_dict.setdefault(s, np.array([])), pm_degree_dict[t])
    pm_neigh_dict[t] = np.append(pm_neigh_dict.setdefault(t, np.array([])), pm_degree_dict[s])
    
for i in range(x1.shape[0]):
    degree_seq = pm_neigh_dict[i]
    deg = pm_degree_dict[i]
    min_deg = np.min(degree_seq)
    max_deg = np.max(degree_seq)
    mean_deg = np.mean(degree_seq)
    std_deg = np.std(degree_seq)
    pm_features[i][0] 
    pm_features[i][1] = min_deg
    pm_features[i][2] = max_deg
    pm_features[i][3] = mean_deg
    pm_features[i][4] = std_deg
    
del(pm_degree_dict)
del(pm_neigh_dict)

x1 = torch.tensor(pm_features).float()


ot_degree_dict = {}
ot_neigh_dict = {}
ot_features = np.empty((x2.shape[0], 5))

for i in range(x2.shape[0]):
    ot_degree_dict.setdefault(i, 0)
    ot_neigh_dict.setdefault(i, 0)
    
for edge in edge_index2.T:
    s, t = list(map(int, edge))
    ot_degree_dict[s] = ot_degree_dict.setdefault(s, 0) + 1
    ot_degree_dict[t] = ot_degree_dict.setdefault(t, 0) + 1

for edge in edge_index2.T:
    s, t = list(map(int, edge))
    ot_neigh_dict[s] = np.append(ot_neigh_dict.setdefault(s, np.array([])), ot_degree_dict[t])
    ot_neigh_dict[t] = np.append(ot_neigh_dict.setdefault(t, np.array([])), ot_degree_dict[s])

for i in range(x2.shape[0]):
    degree_seq = ot_neigh_dict[i]
    deg = ot_degree_dict[i]
    min_deg = np.min(degree_seq)
    max_deg = np.max(degree_seq)
    mean_deg = np.mean(degree_seq)
    std_deg = np.std(degree_seq)
    ot_features[i][0] = deg
    ot_features[i][1] = min_deg
    ot_features[i][2] = max_deg
    ot_features[i][3] = mean_deg
    ot_features[i][4] = std_deg
    
del(ot_degree_dict)
del(ot_neigh_dict)

x2 = torch.tensor(ot_features).float()

#### Hungarian algorithm on topological features

In [30]:
times = []
for i in [10, 100, 300, 500, 1000, 2000, 4000, 8000, 11701]:
    x_pm = x1[:i]
    x_ot = x2[:i]
    sim_eucl = x_pm @ x_ot.T
    sim_cos = pairwise_distances(x_pm, x_ot, metric='cosine')
    start = time()
    match_eucl = linear_sum_assignment(sim_eucl, maximize=True)
    match_cos = linear_sum_assignment(sim_cos)
    end = time()
    times.append(end - start)
    hits_at_1_eucl = (match_eucl[0] == match_eucl[1]).sum() / i
    hits_at_1_cos = (match_cos[0] == match_cos[1]).sum() / i
    print(f'{i} eucl: {hits_at_1_eucl:.04f}, cos: {hits_at_1_cos:.04f}, time: {end - start:.02f}')

10 eucl: 0.2000, cos: 0.1000, time: 0.00
100 eucl: 0.0200, cos: 0.0200, time: 0.00
300 eucl: 0.0000, cos: 0.0033, time: 0.05
500 eucl: 0.0040, cos: 0.0020, time: 0.16
1000 eucl: 0.0020, cos: 0.0010, time: 0.90
2000 eucl: 0.0000, cos: 0.0005, time: 7.76
4000 eucl: 0.0000, cos: 0.0000, time: 74.64
8000 eucl: 0.0000, cos: 0.0001, time: 780.68
11701 eucl: 0.0002, cos: 0.0005, time: 2073.52
